In [246]:
import pandas as pd
import numpy as np
import os
#import chinese_calendar
import matplotlib.pyplot as plt
import datetime
import json
import pickle
import math
import itertools
import time
from ortools.linear_solver import pywraplp
import warnings
warnings.filterwarnings(action='ignore')

In [247]:
os.environ['PYSPARK_PYTHON'] = '/home/luban/anaconda3/envs/python36/bin/python'
import findspark
findspark.init()

from pyspark import *
from pyspark.sql import *

app_name = 'xuyunqing_jupyter'
conf = (SparkConf().setAppName(app_name)
        .set('spark.master', 'yarn')
        .set('spark.submit.deployMode', 'client')
        .set('spark.driver.memory',                  '10g')
        .set('spark.executor.memory',                '6g')
        .set('spark.dynamicAllocation.minExecutors', '2')
        .set('spark.dynamicAllocation.maxExecutors', '100')
        .set('spark.yarn.queue',                     'celuemoxingbu_driver_service_score_orig')
        .set('spark.ui.port','8060')
        .set('spark.executor.memoryOverhead','4096')
        .set("spark.sql.execution.arrow.pyspark.enabled", "true")
        .set('spark.driver.allowMultipleCoo nntexts', 'True')
        .set('spark.yarn.dist.archives','hdfs://DClusterNmg4/user/bigdata_driver_ecosys_test/ly/python_env/python3.6.2.tgz#python3.6.2')
        .set('spark.pyspark.driver.python',          '/home/luban/anaconda3/envs/python36/bin/python')
        .set('spark.pyspark.python', './python3.6.2/bin/python')
        .set("spark.yarn.appMasterEnv.PYSPARK_PYTHON", './python3.6.2/bin/python')
        )
spark = SparkSession.builder.config(conf = conf).enableHiveSupport().getOrCreate()

from pyspark.sql.functions import col,udf,date_sub,explode,split,date_add
from pyspark.sql.types import StringType,DoubleType,IntegerType
import pandas as pd
import numpy as npx
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('expand_frame_repr',False)
import matplotlib.pyplot as plt

In [300]:
cis = [1,7,8,9,13,14,15,16,20,21,22,23,25,26,28,30,32,33,34,35,36,38,39,40,42,43,44,45,46,47,49,52,53,54,55,56,57,58,59,60,61,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,82,85,86,88,89,91,96,98,99,102,103,104,105,107,109,116,117,118,119,120,122,124,125,126,127,128,129,130,131,132,133,135,137,138,140,141,144,145,146,148,149,150,153,154,156,157,158,160,162,163,164,166,168,169,171,174,175,177,178,179,180,181,185,190,191,192,193,194,195,196,197,198,199,200,201,202,203,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,222,224,225,226,228,230,231,232,238,239,242,243,246,247,248,249,251,252,253,254,255,256,258,260,261,262,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,280,281,282,283,284,286,287,290,291,292,293,294,295,296,297,298,299,300,301,302,303,305,306,308,309,310,311,312,313,314,315,316,317,318,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,359,360,361,362,363,364,365,366,367,368,369,370,371,372]
citystr = ','.join(map(str, cis))
alloc_cis = list(set(cis) - set([1,13,16]))

In [302]:
gmv_dt, cr_dt = '2023-05-31',  '2023-05-31'
sdate, edate = "2023-06-01", "2023-06-01"
city_bucket_budget = 4505305

# 函数

In [304]:
def gen_ecr_bucket(x):
    if x <= 0.59:  # x > 0.15
        return 0
    elif x > 0.59 and x <= 0.635:
        return 1
    elif x > 0.635 and x <= 0.66:
        return 2
    elif x > 0.66 and x <= 0.685:
        return 3
    elif x > 0.685: 
        return 4
    
    
def gen_ecr_dgmv_ratio(x):
    w = 0.65
    if x.ecr_bin == 0:
        return 0.744 * (x.fkhf ** (w)) # 0.744
    elif x.ecr_bin == 1:
        return 0.69 * (x.fkhf ** (w)) # 0.69
    elif x.ecr_bin == 2:
        return 0.64 * (x.fkhf ** (w)) # 0.64
    elif x.ecr_bin == 3:
        return 0.597 * (x.fkhf ** (w)) # 0.597
    elif x.ecr_bin == 4:
        return 0.542 * (x.fkhf ** (w)) # 0.542

In [305]:
def gen_ecr_dgmv_ratio_new(x):
    w = 0.65
    if x.ecr_bin == 0:
        return 0.68 * (x.fkhf ** (w)) # 0.744
    elif x.ecr_bin == 1:
        return 0.66 * (x.fkhf ** (w)) # 0.69
    elif x.ecr_bin == 2:
        return 0.64 * (x.fkhf ** (w)) # 0.64
    elif x.ecr_bin == 3:
        return 0.62 * (x.fkhf ** (w)) # 0.597
    elif x.ecr_bin == 4:
        return 0.6 * (x.fkhf ** (w)) # 0.542

In [306]:
def general_enlarge_rest_budget(choose_scheme, budget, scale_cols):
    rest_money = choose_scheme['delta_cost'].sum()
    enlarge_times = budget / rest_money
    print('enlarge_times:', enlarge_times)
    choose_scheme[scale_cols] = choose_scheme[scale_cols].apply(lambda x: x * enlarge_times, axis=1)
    return choose_scheme

In [307]:
sql2 = """
select
    city_id
    ,cast(date_add(dt, 7) as String) stat_date
    ,bubble_cnt
    ,compete_call_cnt
    ,cast(nvl(compete_call_cnt/bubble_cnt, 0) as Double) wyc_ecr
from 
    smt_stg.budget_allocation_xfk_city_daily_table
where dt between date_sub('{start_date}', 7) and date_sub('{end_date}', 7) and city_id in ({city_list})
    and product_line='网约车'
    and group_name='rgroup_joint_blank'
"""

In [308]:
pred_cr_sql = '''
select
    a.city_id
    ,gmv_type
    ,exp_cr as score
    ,dt
    ,stat_date
from
    (select  
            cast(city_id as Integer) city_id
            ,half_hour
            ,cast(nvl(finish_rate, 0) as Double) exp_cr
            ,dt as stat_date
            ,concat_ws('-', year, month, day) as dt
    from    riemann.gongxu_cr_half_hour_pred_test
    where   exp_id = 'irving'
      and   product_id = 'day_110100_110400_110800_110103'
      and concat_ws('-', year, month, day)=date_sub('{start_date}', 1)
      and city_id in ({city_list})
    )a
    join
    (select
        distinct city_id
        ,gmv_type
    from
        smt_stg.smt_err_budget_alloc_city_t1
    where
        dt='2023-04-01'
        and city_type='非战区非下沉'
    )b
    on a.city_id=b.city_id
'''.format(cr_dt=cr_dt, start_date=sdate, city_list=citystr)

In [309]:
pred_cr_sql = '''
select
    estimate_date as stat_date
    ,city_id
    ,cast(sum(if(key_id='objective_exp_openapi_pp', estimate_value, 0)) as Double) score -- 剔除openapi的客观cr
from 
    mp_data.app_trip_mkt_supply_demand_forecast_result_di
where
    dt = date_sub('{cr_dt}', 0) -- 指定的 gmv dt 分区
    and  estimate_date between '{start_date}' and '{end_date}'
    and city_id in ({city_list})
    and product_name = '泛快2（快车&特惠快车&市内拼车）'
group by estimate_date, city_id
'''.format(cr_dt=cr_dt, start_date=sdate,end_date = edate, city_list=citystr)

In [310]:
mps_candidate_sql = """
    select
        base.tdate stat_date,
        base.city_id city_id,
        base.combo_idx combo_idx,
        base.combo_subsidy_rate combo_subsidy_rate,
        is_candidate,
        cast(coalesce(pred_roi * pred_delta_cost,0) as Double) delta_gmv,
        cast(coalesce(pred_profit_roi * pred_delta_cost,0) as Double) delta_bp,
        cast(coalesce(pred_delta_cost,0) as Double) delta_cost,
        concat_ws(':',cast(base.city_id as  string),  base.combo_subsidy_rate) robot_k,
        cast(split(base.combo_subsidy_rate, ',')[0] as double) as kc_subsidy_rate,
        cast(split(base.combo_subsidy_rate, ',')[1] as double) as th_subsidy_rate,
        cast((pukuai_gmv*cast(split(base.combo_subsidy_rate, ',')[0] as double) + tehui_gmv * cast(split(base.combo_subsidy_rate, ',')[1] as double))/total_gmv as Double) crate,
        cast(total_gmv as DOUBLE) total_gmv, 
        cast(pukuai_gmv as DOUBLE) pukuai_gmv, 
        cast(tehui_gmv as DOUBLE) tehui_gmv
        from
        (select
            city_id,
            combo_idx,
            combo_subsidy_rate,
            is_candidate,
            tdate
        from
            smt_stg.budget_allocation_pas_candidate
        where
            concat_ws('-',year,month,day)='{activity_date}' and tdate between '{start_date}' and '{end_date}' 
            and city_id in ({city_list}) and version = 'v2'
        ) base
        left join
        (select *
         from
            (select
                city_id,
                combo_idx,
                combo_subsidy_rate,
                pred_roi,
                cast(concat_ws('-',year,month,day) as String) tdate
            from
                smt_stg.budget_allocation_pred_delta_gmv_v3
            where
                concat_ws('-',year,month,day)='{activity_date}' and city_id in ({city_list}) and version = 'roi_lgb_v1'
            )
            union
            (select
                city_id,
                combo_idx,
                combo_subsidy_rate,
                pred_roi,
                cast(date_add(concat_ws('-',year,month,day), 7) as String) tdate
            from
                smt_stg.budget_allocation_pred_delta_gmv_v3
            where
                concat_ws('-',year,month,day) between date_sub(date_add('{activity_date}', 1), 7) and date_sub('{end_date}', 7) 
                and city_id in ({city_list}) and version = 'roi_lgb_v1'
            )        
        ) gmv
        on base.city_id = gmv.city_id and base.combo_idx = gmv.combo_idx and base.combo_subsidy_rate=gmv.combo_subsidy_rate and base.tdate = gmv.tdate
        left join
        (select *
        from
            (select
                city_id,
                combo_idx,
                combo_subsidy_rate,
                pred_profit_roi,
                cast(concat_ws('-',year,month,day) as String) tdate
            from
                smt_stg.budget_allocation_pred_delta_profit_ivf
            where
                concat_ws('-',year,month,day)='{activity_date}' and city_id in ({city_list}) and version = 'iv_forest_v1'
            )
            union
            (select
                city_id,
                combo_idx,
                combo_subsidy_rate,
                pred_profit_roi,
                cast(date_add(concat_ws('-',year,month,day), 7)  as String) tdate
            from smt_stg.budget_allocation_pred_delta_profit_ivf
            where
                concat_ws('-',year,month,day) between date_sub(date_add('{activity_date}', 1), 7) and date_sub('{end_date}', 7) 
                and city_id in ({city_list}) and version = 'iv_forest_v1'
            )
        ) profit
        on base.city_id = profit.city_id and base.combo_idx = profit.combo_idx and base.combo_subsidy_rate = profit.combo_subsidy_rate and base.tdate = profit.tdate
        left join
        (select
            city_id,
            combo_idx,
            combo_subsidy_rate,
            pred_delta_cost,
            tdate,
            pukuai_gmv,
            tehui_gmv
        from
            smt_stg.budget_allocation_pred_delta_cost
        where
            concat_ws('-',year,month,day)='{activity_date}' and tdate between '{start_date}' and '{end_date}'   
            and city_id in ({city_list}) and version = 'v1'
        ) cost
        on base.city_id = cost.city_id and base.combo_idx = cost.combo_idx and base.combo_subsidy_rate=cost.combo_subsidy_rate and base.tdate = cost.tdate
        left join
        ( select
            estimate_date as pred_date,
            city_id,
            sum(case when key_id = 'gmv' then estimate_value else 0.0 end ) as total_gmv,
            coalesce( sum( case when key_id = 'gmv' then estimate_value else 0.0 end ) / sum( case when key_id = 'finish_order_cnt' then estimate_value else 0.0 end ),10.0) as asp,
            sum(case when key_id = 'total_tsh' then estimate_value else 0.0 end)/sum( case when key_id = 'compete_call_cnt' then estimate_value else 0.0 end ) as dsr
        from
            mp_data.app_trip_mkt_supply_demand_forecast_result_di
        where
            dt = '{gmv_dt}' -- 指定的 gmv dt 分区
            and estimate_date between '{start_date}' and '{end_date}'
            and product_name = '泛快含优不含拼（快车（普通快车、A+、车大、巡游网约车）、优享、D1、特惠快车、涅槃）' --新泛快
        group by
            estimate_date,
            city_id) dsr
     on base.city_id = dsr.city_id and base.tdate= dsr.pred_date
    """

In [311]:
class BaseSolver:
    """简化版预算分配求解器"""

    def __init__(self, candidates, is_bp=True, solver_id='CBC', city_must_cost=True, mode='all', total_budget=0, other_constraints=[]):
        self.solver = pywraplp.Solver.CreateSolver(solver_id)
        if mode == 'all':
            self.budget = 23455199
        elif mode == 'city_bucket':
            self.budget = total_budget
        self.other_constraints = other_constraints
        self.is_bp = is_bp
        self.df = candidates
        self.df = candidates[candidates[Valid_Col] == 1]
        self.city_must_spend = city_must_cost
        self.xs = None

    def create_variables(self):
        self.xs = [self.solver.IntVar(0, 1, f'combo_{i}') for i in self.df.index]

    def add_constraints(self):
        """添加求解约束
        目前只有两种约束：预算约束、分城市给补
        """
        cost = self.solver.Sum(self.xs[i] * self.df.loc[i, 'delta_cost'] for i in self.df.index)
        self.solver.Add(cost <= self.budget)
        self.df.groupby(['city_id', 'stat_date']).apply(self.add_city_constraint)
        '''自定义约束'''
        for cons in self.other_constraints:
            print("自定义约束：", cons)
            if cons == 'pk_budget_upper_limit':
                self.add_pk_constarint()

    def add_pk_constarint(self, ratio=0.025):
        day_num = len(self.df.stat_date.unique())
        day_pk_budget = float(self.budget * ratio / day_num)
        print("分框天数:", day_num, "day pukuai budget:", day_pk_budget)
        self.df.groupby(['stat_date']).apply(self.add_day_pk_constraint, day_pk_budget=day_pk_budget)

    def add_day_pk_constraint(self, group_df, day_pk_budget):
        pk_cost = self.solver.Sum(self.xs[i] * self.df.loc[i, 'pk_budget'] for i in group_df.index)
        self.solver.Add(pk_cost >= day_pk_budget)

    def set_objective(self):
        dgmv = self.solver.Sum(self.xs[i] * self.df.loc[i, 'ate'] for i in self.df.index)
        self.solver.Maximize(dgmv)

    def cal_ate(self, lambda_value):
        if self.is_bp:
            self.df['ate'] = self.df['delta_gmv'] + lambda_value * self.df['delta_bp']
        else:
            self.df['ate'] = self.df['delta_gmv']

    def solve(self):
        # 确保 index 为 0 到 length - 1
        lambda_value = self.get_lambda([2, 1.5, 1, 0.5, 0.4, 0.3, 0.25, 0.2, 0.15, 0.1, 0.05, 0.01])
        self.cal_ate(lambda_value)
        self.df = self.df.reset_index()
        self.create_variables()
        self.add_constraints()
        self.set_objective()
        start = time.time()
        status = self.solver.Solve()
        print(f'Solve status: {status}. Time cost: {time.time() - start:.2f}s')
        out_put = self.return_candicate(status)
        return out_put

    def return_candicate(self, status):
        if status == 0:
            index_list = []
            for index in range(len(self.xs)):
                if self.xs[index].solution_value() > 0:
                    index_list.append(index)
            out_put = self.df.loc[index_list, ]
            print(out_put.delta_cost.sum())
            return out_put
        else:
            return pd.DataFrame()


    def add_city_constraint(self, group_df):
        expr = self.solver.Sum(self.xs[i] for i in group_df.index)
        if self.city_must_spend:
            self.solver.Add(expr == 1)
        else:
            self.solver.Add(expr <= 1)


    def get_lambda(self, lambda_list):
        """
        基于预算包选择合适的lambda，筛选出每个城市符合条件的c补组合对应的delta_ate最大的行对应的cost累计，来看最大cost
        """
        if not self.is_bp:
            return 0
        for l in lambda_list:
            total_cost = 0
            self.df['ate'] = self.df['delta_gmv'] + l * self.df['delta_bp']
            for city_id in self.df['city_id'].unique():
                tmp = self.df[self.df['city_id'] == city_id]
                tmp_sort = tmp.sort_values(by='ate', ascending=False).reset_index()
                if tmp_sort.loc[0, 'ate'] > 0:
                    cost = tmp_sort.loc[0, 'delta_cost']
                    total_cost += cost
            if total_cost >= self.budget * 1.1:
                return l
        return l

# 模拟

In [312]:
sql = mps_candidate_sql.format(activity_date=sdate,
                                       gmv_dt=gmv_dt,
                                    start_date=sdate,
                                    end_date=edate,
                                    city_list=citystr)
print(sql)
pred_candidate = spark.sql(sql).toPandas()

pred_candidate['pk_budget'] = pred_candidate['pukuai_gmv'] * pred_candidate['kc_subsidy_rate']
pred_candidate['th_budget'] = pred_candidate['tehui_gmv'] * pred_candidate['th_subsidy_rate']
pred_candidate['city_day_key'] = pred_candidate.apply(lambda x: str(x.city_id)+'_' + x.stat_date,axis = 1)

lack_cis = pred_candidate[pred_candidate['delta_gmv'].isna()].city_id.unique()
print("delta_gmv null的城市:", lack_cis)
print("delta_gmv null的日期:", pred_candidate[pred_candidate['delta_gmv'].isna()].stat_date.unique())

gmvdf = pred_candidate[['city_id', 'stat_date','total_gmv' ]].drop_duplicates()


    select
        base.tdate stat_date,
        base.city_id city_id,
        base.combo_idx combo_idx,
        base.combo_subsidy_rate combo_subsidy_rate,
        is_candidate,
        cast(coalesce(pred_roi * pred_delta_cost,0) as Double) delta_gmv,
        cast(coalesce(pred_profit_roi * pred_delta_cost,0) as Double) delta_bp,
        cast(coalesce(pred_delta_cost,0) as Double) delta_cost,
        concat_ws(':',cast(base.city_id as  string),  base.combo_subsidy_rate) robot_k,
        cast(split(base.combo_subsidy_rate, ',')[0] as double) as kc_subsidy_rate,
        cast(split(base.combo_subsidy_rate, ',')[1] as double) as th_subsidy_rate,
        cast((pukuai_gmv*cast(split(base.combo_subsidy_rate, ',')[0] as double) + tehui_gmv * cast(split(base.combo_subsidy_rate, ',')[1] as double))/total_gmv as Double) crate,
        cast(total_gmv as DOUBLE) total_gmv, 
        cast(pukuai_gmv as DOUBLE) pukuai_gmv, 
        cast(tehui_gmv as DOUBLE) tehui_gmv
        from
        (select
   

delta_gmv null的城市: []
delta_gmv null的日期: []


In [313]:
ecrdf1 = spark.sql(sql2.format(start_date=sdate, end_date=edate, city_list=citystr)).toPandas()
ecrdf1 = ecrdf1.query("city_id in @cis and stat_date >= '%s' and stat_date <= '%s'" % (sdate, edate))

In [314]:
from datetime import datetime, timedelta
day1 = (datetime.strptime(sdate, "%Y-%m-%d") - timedelta(days=90)).strftime("%Y-%m-%d")
print("开始时间:", day1)
past_day_sql = sql2.format(start_date=day1, end_date=edate, city_list=citystr)
pastdf = spark.sql(past_day_sql).toPandas()


df_li = []
for day in pd.date_range(sdate, edate).astype(str):
    tt = ecrdf1.query("stat_date=='%s'" % day)
    city_list = alloc_cis
    lack_cis = list(set(city_list) - set(tt.city_id.unique()))
    print(day, "当日缺失城市：", lack_cis)
    filldf = pastdf.query("city_id in @lack_cis").groupby('city_id')[['wyc_ecr']].agg('mean').reset_index()
    filldf['stat_date'] = day
    df_li.append(filldf)

filldf = pd.concat(df_li, ignore_index=True)
print("填充行数:", len(filldf))
ecrdf = pd.concat([ecrdf1, filldf], ignore_index=True)[['city_id', 'stat_date', 'wyc_ecr']]

开始时间: 2023-03-03
2023-06-01 当日缺失城市： []
填充行数: 0


In [315]:
ecrdf['ecr_bin'] = ecrdf['wyc_ecr'].apply(gen_ecr_bucket)
ecrdf = ecrdf[~ecrdf['ecr_bin'].isna()]

dates = list(pd.date_range(sdate, edate).astype(str))
fkhf_list = list(itertools.product(cis, dates, np.arange(0, 0.11, 0.0005)))
ratedf = pd.DataFrame(fkhf_list, columns=['city_id', 'stat_date', 'fkhf'])
print("ratedf:", ratedf.head(5))

ratedf:    city_id   stat_date    fkhf
0        1  2023-06-01  0.0000
1        1  2023-06-01  0.0005
2        1  2023-06-01  0.0010
3        1  2023-06-01  0.0015
4        1  2023-06-01  0.0020


In [316]:
df = pd.merge(ecrdf, ratedf, on=['city_id', 'stat_date'])
df['ecr_dgmv_ratio'] = df.apply(gen_ecr_dgmv_ratio, axis=1)
# df['ecr_dgmv_ratio'] = df.apply(gen_ecr_dgmv_ratio_new, axis=1)

In [317]:
opti_data = pd.merge(df, gmvdf, on=['city_id', 'stat_date'], how='left')

opti_data['delta_gmv'] = opti_data['ecr_dgmv_ratio'] * opti_data['total_gmv']
opti_data['delta_cost'] = opti_data['fkhf'] * opti_data['total_gmv']

In [318]:
opti_data2 = opti_data.query("city_id in @alloc_cis")
opti_data2['is_candidate']=1

In [319]:
Valid_Col = 'is_candidate'
opti_data[Valid_Col] = 1
solver = BaseSolver(opti_data2, is_bp=False, city_must_cost=True, mode='city_bucket', total_budget=city_bucket_budget)

In [320]:
resdf = solver.solve()

Solve status: 0. Time cost: 0.44s
4505295.19661196


In [322]:
bin_costdf = resdf.groupby(['ecr_bin'])[['delta_cost', 'total_gmv']].agg(sum).reset_index().rename(columns={'delta_cost': 'budget'})
print("ecr分框的预算:")
print(bin_costdf)

ecr分框的预算:
   ecr_bin        budget     total_gmv
0        0  4.474147e+05  6.883303e+06
1        1  1.642323e+06  3.122939e+07
2        2  6.391007e+05  1.524255e+07
3        3  2.553315e+05  7.400912e+06
4        4  1.521125e+06  5.751086e+07


In [323]:
bin_costdf['avg_rate'] = bin_costdf['budget']/bin_costdf['total_gmv']

In [324]:
bin_costdf

,ecr_bin,budget,total_gmv,avg_rate
0,0,4.474147e+05,6.883303e+06,0.065000
1,1,1.642323e+06,3.122939e+07,0.052589
2,2,6.391007e+05,1.524255e+07,0.041929
3,3,2.553315e+05,7.400912e+06,0.034500
4,4,1.521125e+06,5.751086e+07,0.026449


# cr分框

In [325]:
crdf = spark.sql(pred_cr_sql).toPandas()

score = crdf.query("stat_date >= '%s' and stat_date <= '%s'" % (sdate, edate))

In [326]:
pred_cr_sql

"\nselect\n    estimate_date as stat_date\n    ,city_id\n    ,cast(sum(if(key_id='objective_exp_openapi_pp', estimate_value, 0)) as Double) score -- 剔除openapi的客观cr\nfrom \n    mp_data.app_trip_mkt_supply_demand_forecast_result_di\nwhere\n    dt = date_sub('2023-05-31', 0) -- 指定的 gmv dt 分区\n    and  estimate_date between '2023-06-01' and '2023-06-01'\n    and city_id in (1,7,8,9,13,14,15,16,20,21,22,23,25,26,28,30,32,33,34,35,36,38,39,40,42,43,44,45,46,47,49,52,53,54,55,56,57,58,59,60,61,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,82,85,86,88,89,91,96,98,99,102,103,104,105,107,109,116,117,118,119,120,122,124,125,126,127,128,129,130,131,132,133,135,137,138,140,141,144,145,146,148,149,150,153,154,156,157,158,160,162,163,164,166,168,169,171,174,175,177,178,179,180,181,185,190,191,192,193,194,195,196,197,198,199,200,201,202,203,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,222,224,225,226,228,230,231,232,238,239,242,243,246,247,248,249,251,252,253,254,255,256,258,260,2

In [327]:
def gen_cr_bucket2(x):
    if x <= 0.7:
        return 0
    elif x>0.7 and x <= 0.8:
        return 1
    elif x > 0.8 and x <= 0.86:
        return 2
    elif x > 0.86 and x <= 0.9:
        return 3
    elif x > 0.9 and x <= 0.93:
        return 4
    else:
        return 5
def gen_cr_bucket(x):
    if x > 0 and x <= 0.5:
        return 0
    elif x > 0.5 and x <= 0.6:
        return 1
    elif x > 0.6 and x <= 0.68:
        return 2
    elif x > 0.68 and x <= 0.72:
        return 3
    elif x > 0.72 and x <= 0.76:
        return 4
    elif x > 0.76 and x <= 0.8:
        return 5
    elif x > 0.8 and x <= 0.85:
        return 6
    elif x > 0.85:
        return 7

In [328]:
# def gen_cr_bucket1(x, m):
#     if x <= m * 0.95:
#         return 0
#     elif x > m * 0.95 and x <= m * 0.97:
#         return 1
#     elif x > m * 0.97 and x <= m:
#         return 2
#     elif x > m and x <= m * 1.03:
#         return 3
#     elif x > m*1.03 and x <= m * 1.05:
#         return 4
#     else:
#         return 5

# cr_col = 'score'
# score = score[score[cr_col] > 0]
# score['bin'] = score[cr_col].apply(gen_cr_bucket2)
# score['bin_val'] = score['bin'].apply(func_val)

In [329]:
cr_col = 'score'
score = score[score[cr_col] > 0]
score['bin'] = score[cr_col].apply(gen_cr_bucket)
score['bin_val'] = score['bin'].apply(lambda x: (x + 1) * 0.05 if x <= 3 else (0.25 * (x + 1) if x==4 else 0.35 * x))

In [330]:
weightdf = pd.merge(score, gmvdf, on=['city_id', 'stat_date']).query("city_id in @alloc_cis")
weightdf['weight'] = weightdf.apply(lambda x: x['total_gmv'] * x['bin_val'], axis=1)

df_li = []
ecr_bins = sorted(ecrdf.ecr_bin.unique())
for ecr_bin in ecr_bins:
    tmpdf = ecrdf.query("ecr_bin==%d and city_id in @alloc_cis" % ecr_bin)[['city_id', 'stat_date']]
    tmpdf2 = pd.merge(weightdf, tmpdf)
    print(len(tmpdf2))
    if len(tmpdf2) > 0:
        bin_budget = bin_costdf.query("ecr_bin==%d" % ecr_bin)['budget'].iloc[0] # ecr分框预算
        avg_rate = (bin_budget) / (tmpdf2['weight'].sum())  # 泛快呼返
        print("avg_rate:", avg_rate)
        tmpdf2['avg_rate'] = tmpdf2['bin_val'].apply(lambda x: avg_rate * x)
        tmpdf2['ecr_bin'] = ecr_bin
        df_li.append(tmpdf2)
    else:
        continue
cr_ratedf = pd.concat(df_li, ignore_index=True)

4
avg_rate: 0.030208048565663177
7
avg_rate: 0.022784563798036377
4
avg_rate: 0.017469507162270283
2
avg_rate: 0.014081632653061227
18
avg_rate: 0.011131294977382567


In [331]:
'''过滤候选集'''
cr_ratedf['min_combo_rate'] = cr_ratedf['avg_rate'].apply(lambda x: x * 0.8).astype(float)
cr_ratedf['max_combo_rate'] = cr_ratedf['avg_rate'].apply(lambda x: x * 1.2).astype(float)

# 临时兜底约束
# cr_ratedf['max_combo_rate'] = cr_ratedf.apply(lambda x: 0.01 if x.city_id in (8, 25) else x.max_combo_rate, axis=1)
# cr_ratedf['min_combo_rate'] = cr_ratedf.apply(lambda x: 0.0 if x.city_id in (8, 25) else x.min_combo_rate, axis=1)
# cr_ratedf['max_combo_rate'] = cr_ratedf.apply(lambda x: 0.02 if x.city_id in (14, 16, 22) else x.max_combo_rate, axis=1)
# cr_ratedf['min_combo_rate'] = cr_ratedf.apply(lambda x: 0.0 if x.city_id in (14, 16, 22) else x.min_combo_rate, axis=1)


                
group_candidate = cr_ratedf.merge(pred_candidate, on=['city_id', 'stat_date'], how='left')
print("城市分框，候选集数量：", len(group_candidate))
filter_pred_candidate = group_candidate.query("is_candidate==1 and crate>=min_combo_rate and crate<=max_combo_rate")
print("城市分框，过滤后的候选集数量：", len(filter_pred_candidate))

城市分框，候选集数量： 55965
城市分框，过滤后的候选集数量： 4708


In [341]:
filter_pred_candidate['city_id'].unique().shape

(35,)

# 求解

In [345]:

solverold = BaseSolver(filter_pred_candidate, is_bp=True, city_must_cost=True,mode='city_bucket', total_budget=city_bucket_budget, other_constraints=['pk_budget_upper_limit'])
resdfold = solverold.solve()
scale_cols = ['kc_subsidy_rate', 'th_subsidy_rate', 'pk_budget', 'th_budget', 'delta_cost']
resdfold2 = general_enlarge_rest_budget(resdfold, city_bucket_budget, scale_cols)
resdfold2['ate_roi'] = resdfold2['ate']/resdfold2['delta_cost']

自定义约束： pk_budget_upper_limit
分框天数: 1 day pukuai budget: 112632.625
Solve status: 0. Time cost: 0.74s
4505304.153910066
enlarge_times: 1.0000001877986269
